# Processing image and volume crops
Here we will create simple tools for analysis of existing crops:
- given full input volume and a crop find its position in it and save to file
- given position of the crops, crop the original (handle multi channel volume)
- validate created crops


### Contents:
1. Loading and setting up data paths
2. Show cropping in 2d and 3d
3. Run info extractor on all existing annotations
4. Validate cropping information

In [ ]:
from notebook_utils import *

import imageio
import skimage.segmentation
import skimage.filters

import scipy.stats

import random

import os
import shutil

from tqdm import tqdm as tqdm

%matplotlib inline

## Setup paths and common methods

In [ ]:
import clb.utils

In [ ]:
data_root = r'D:\Fafa\MIT\CellDx'

# Cropping in 2d and 3d

## Load input data

In [ ]:
dir_root = r'D:\Fafa\MIT\CellDx\_finding_crops'
path_3d = os.path.join(dir_root, 'full_volume.tif')
full = clb.utils.normalize_channels_volume(imageio.volread(path_3d))

In [ ]:
path_3d = os.path.join(dir_root, 'substack.tif')
cp_segm_mult_channel = imageio.volread(path_3d) 
cp_segm_3 = imageio.volread(path_3d)[::,::,::,2].copy()
show_all(1,2,cp_segm_3[0], cp_segm_3[-5], scale=5)

In [ ]:
path_crop = os.path.join(dir_root, '#8T 2048_1024 crop_2 slim_dapi.tif')
crop_volume = clb.utils.normalize_channels_volume(imageio.volread(path_crop))

path_crop = os.path.join(dir_root, 'mini_crop_20.tif')
crop = imageio.volread(path_crop)[::,::,2]
show_all(1,2,crop_volume[0], crop, scale=5)

### Sample cropping

In [ ]:
single = cp_segm_3[0]
crop_in_single = single[30:230,50:250]
single_marked = single.copy()
single_marked[30:230,50:250] = 255*255

import clb.image_processing
#correlation2 = correlate2d(single_small, crop_in_single_small - crop_in_single_small.mean(), boundary='symm', mode='same')
correlation2 = clb.image_processing.correlate2d_with_fft(single, crop_in_single - crop_in_single.mean())
show_all(1,2,single_marked, correlation2, scale=10)

### Find cropped volume in 3D

In [ ]:
import clb.cropping
importlib.reload(clb.cropping)

results = clb.cropping.find_positions_3d(full, crop_volume)

In [ ]:
crop_middle = crop_volume[2]
crop_info_middle = results[1][2]
recroped = crop_info_middle.crop(full)
show_all(1,2, crop_middle, recroped, scale=5)

# Run info extractor on all existing annotations

In [ ]:
import clb.evaluator
importlib.reload(clb.evaluator)
importlib.reload(clb.cropping)
run_all = False

In [ ]:
if run_all:
    path_3d = os.path.join(dir_root, 'T8 S2 C1 -#8T 2048 1024 0.5um more images.tif')
    full = clb.utils.normalize_channels_volume(imageio.volread(path_3d))
datasets = get_standard_datasets(r"G:\Dyski zespołu\CLDX\Pilot\Private\_annotations", r'T8_S2')

In [ ]:
for d in tqdm(datasets):
    if run_all:
        crop_volume = imageio.volread(d.input)
        one_channel_crop_volume = clb.utils.normalize_channels_volume(crop_volume)
        qual, pos = clb.cropping.find_positions_3d(full, one_channel_crop_volume)
        yaml_path = d.input[:-4] + ".yaml"
        print (d.input, "=", qual)
        clb.cropping.CropInfo.save(pos, yaml_path)

# Validate cropping information

In [ ]:
run_validate_cropping = False

In [ ]:
if run_validate_cropping:
    # 'T8 S2 C1 -#8T 2048 1024 0.5um more images.tif'
    path_3d = os.path.join(data_root, 'C1-#8T S3 2048 1024 0.5um more images.lif - #8T 2048 step 0.5um-1.tif')
    full = clb.utils.normalize_channels_volume(imageio.volread(path_3d))
    datasets = get_standard_datasets(r"G:\Dyski zespołu\CLDX\Pilot\Private\_annotations", r'T8_S2')

In [ ]:
import imageio
if run_validate_cropping:
    validation_dir = os.path.join(dir_root, r'verify_s2')
    for d in tqdm(datasets):
        crop_volume = imageio.volread(d.input)
        one_channel_crop_volume = clb.utils.normalize_channels_volume(crop_volume)
        path_yaml = d.input[:-3] + "yaml"
        info = clb.cropping.CropInfo.load(path_yaml)

        filename = os.path.basename(d.input)
        print(filename)
        if len(crop_volume) != len(info):
            raise Exception("Invalid length")

        imageio.imsave(os.path.join(validation_dir, filename), crop_volume[0])
        imageio.imsave(os.path.join(validation_dir, filename + "_re.tif"), info[0].crop(full))